In [11]:
ls ../data/error-analysis/charword-bi-lstm/

error-analysis-train.csv  error-analysis-trial.csv


In [12]:
import pandas as pd

train = pd.read_csv('../data/error-analysis/charword-bi-lstm/error-analysis-train.csv')
trial = pd.read_csv('../data/error-analysis/charword-bi-lstm/error-analysis-trial.csv')

In [13]:
trial.head()

,item-id,text,pred-offset,truth-offset,truth-taboo-words,candid-word,toxic-proba,normal-proba,prediction,grand-truth
0,0,Because he's a moron and a bigot. It's not any...,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",moron bigot,Because,0.049550,0.896776,normal,normal
1,0,Because he's a moron and a bigot. It's not any...,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",moron bigot,he,0.039203,0.929156,normal,normal
2,0,Because he's a moron and a bigot. It's not any...,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",moron bigot,a,0.079390,0.881001,normal,normal
3,0,Because he's a moron and a bigot. It's not any...,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",moron bigot,moron,0.833894,0.122758,toxic,toxic
4,0,Because he's a moron and a bigot. It's not any...,"[15, 16, 17, 18, 19]","[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",moron bigot,and,0.059289,0.922881,normal,normal


In [14]:
from sklearn.metrics import classification_report

In [15]:
#trial F1-tsp 0.55
print(classification_report(trial['grand-truth'], trial['prediction']))

              precision    recall  f1-score   support

      normal       0.95      0.98      0.97     22300
       toxic       0.60      0.37      0.46      1694

    accuracy                           0.94     23994
   macro avg       0.78      0.68      0.71     23994
weighted avg       0.93      0.94      0.93     23994



In [16]:
#train F1-tsp 0.59
print(classification_report(train['grand-truth'], train['prediction']))

              precision    recall  f1-score   support

      normal       0.96      0.99      0.97    260189
       toxic       0.78      0.53      0.63     23241

    accuracy                           0.95    283430
   macro avg       0.87      0.76      0.80    283430
weighted avg       0.94      0.95      0.95    283430



# Trial

In [7]:
toxic = trial[trial['grand-truth']=='toxic']
toxic_pred_normal = toxic[toxic['prediction']=='normal']
print("Grand Truth is toxic: ", toxic.shape[0])
print("Grand Truth is toxic but predicted as Normal", toxic_pred_normal.shape[0])
#1694

print("-----------------------------------------------")
normal = trial[trial['grand-truth']=='normal']
normal_pred_toxic = normal[normal['prediction']=='toxic']
print("Grand Truth is normal:", normal.shape[0])
print("Grand Truth is normal but predicted as toxic:", normal_pred_toxic.shape[0])
#22300

print("-----------------------------------------------")
predicted_toxic = trial[trial['prediction']=='toxic']
pred_toxic_is_toxic = predicted_toxic[predicted_toxic['grand-truth']=='toxic']
print("Predicted as toxic: ", predicted_toxic.shape[0])
print("Predicted as toxic is actually toxic:", pred_toxic_is_toxic.shape[0])

print("-----------------------------------------------")
predicted_normal = trial[trial['prediction']=='normal']
pred_normal_is_normal = predicted_normal[predicted_normal['grand-truth']=='normal']
print("Predicted as normal: ", predicted_normal.shape[0])
print("Predicted as normal is actually normal:", pred_normal_is_normal.shape[0])

Grand Truth is toxic:  1694
Grand Truth is toxic but predicted as Normal 1066
-----------------------------------------------
Grand Truth is normal: 22300
Grand Truth is normal but predicted as toxic: 416
-----------------------------------------------
Predicted as toxic:  1044
Predicted as toxic is actually toxic: 628
-----------------------------------------------
Predicted as normal:  22950
Predicted as normal is actually normal: 21884


# Train

In [8]:
toxic = train[train['grand-truth']=='toxic']
toxic_pred_normal = toxic[toxic['prediction']=='normal']
print("Grand Truth is toxic: ", toxic.shape[0])
print("Grand Truth is toxic but predicted as Normal", toxic_pred_normal.shape[0])
#23241

print("-----------------------------------------------")
normal = train[train['grand-truth']=='normal']
normal_pred_toxic = normal[normal['prediction']=='toxic']
print("Grand Truth is normal:", normal.shape[0])
print("Grand Truth is normal but predicted as toxic:", normal_pred_toxic.shape[0])
#260189
print("-----------------------------------------------")
predicted_toxic = train[train['prediction']=='toxic']
pred_toxic_is_toxic = predicted_toxic[predicted_toxic['grand-truth']=='toxic']
print("Predicted as toxic: ", predicted_toxic.shape[0])
print("Predicted as toxic is actually toxic:", pred_toxic_is_toxic.shape[0])

print("-----------------------------------------------")
predicted_normal = train[train['prediction']=='normal']
pred_normal_is_normal = predicted_normal[predicted_normal['grand-truth']=='normal']
print("Predicted as normal: ", predicted_normal.shape[0])
print("Predicted as normal is actually normal:", pred_normal_is_normal.shape[0])

Grand Truth is toxic:  23241
Grand Truth is toxic but predicted as Normal 10864
-----------------------------------------------
Grand Truth is normal: 260189
Grand Truth is normal but predicted as toxic: 3451
-----------------------------------------------
Predicted as toxic:  15828
Predicted as toxic is actually toxic: 12377
-----------------------------------------------
Predicted as normal:  267602
Predicted as normal is actually normal: 256738


# AUC

In [9]:
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn import metrics

# TP, prediction is Positive, and this prediction is True
# FP, prediction is Positive, and this prediction is False

# Positive is normal
# Negative is toxic

print("Train:\n", confusion_matrix(train['grand-truth'], train['prediction'], labels=['toxic', 'normal']))

tn, fp, fn, tp = confusion_matrix(train['grand-truth'], train['prediction'], labels=['toxic', 'normal']).ravel()
print("\nTN:{},  FP:{},  FN:{}, TP:{} ".format(tn, fp, fn, tp))

normal = trial['normal-proba'].tolist()
y = [1 if label=='normal' else 0 for label in trial['grand-truth'].tolist()]
print("\nTrial AUC:", metrics.roc_auc_score(y, normal))

print("------------------------------------------------------------------")

print("Trial:\n", confusion_matrix(trial['grand-truth'], trial['prediction'], labels=['toxic', 'normal']))

tn, fp, fn, tp = confusion_matrix(trial['grand-truth'], trial['prediction'], labels=['toxic', 'normal']).ravel()
print("\nTN:{},  FP:{},  FN:{}, TP:{} ".format(tn, fp, fn, tp))

normal = train['normal-proba'].tolist()
y = [1 if label=='normal' else 0 for label in train['grand-truth'].tolist()]
print("\nTrian AUC:", metrics.roc_auc_score(y, normal))


Train:
 [[ 12377  10864]
 [  3451 256738]]

TN:12377,  FP:10864,  FN:3451, TP:256738 

Trial AUC: 0.8135428656137991
------------------------------------------------------------------
Trial:
 [[  628  1066]
 [  416 21884]]

TN:628,  FP:1066,  FN:416, TP:21884 

Trian AUC: 0.9291953195328517


In [10]:
toxic = train[train['grand-truth']=='toxic']
toxic_pred_normal = toxic[toxic['prediction']=='normal']
print("Grand Truth is toxic: ", toxic.shape[0])
print("Grand Truth is toxic but predicted as Normal", toxic_pred_normal.shape[0])
#23241

Grand Truth is toxic:  23241
Grand Truth is toxic but predicted as Normal 10864


In [11]:
#print(toxic_pred_normal['candid-word'].tolist())

In [12]:
index = 9
#df = train
df = trial
display(df[df['item-id']==index])

,item-id,text,pred-offset,truth-offset,truth-taboo-words,candid-word,toxic-proba,normal-proba,prediction,grand-truth
283,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,You,0.011737,0.969474,normal,normal
284,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,wrong,0.013733,0.959585,normal,normal
285,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,The,0.012513,0.963848,normal,normal
286,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,delay,0.009607,0.972878,normal,normal
287,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,between,0.007095,0.979091,normal,normal
288,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,retirement,0.011531,0.962139,normal,normal
289,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,and,0.011663,0.964562,normal,normal
290,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,lobbying,0.029515,0.908894,normal,normal
291,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,is,0.014994,0.959876,normal,normal
292,9,You're wrong. The delay between retirement an...,[],"[277, 278, 279, 280, 281, 282, 283, 284, 285, ...",Don't make ignorant statements,nowhere,0.023970,0.945468,normal,normal
